<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-demo/blob/main/07_sequentialChain_tools_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq openai langchain tiktoken colab-env pymupdf faiss-cpu google-search-results
# !pip install -Uqq pyspark-ai pyspark
!pip install -Uqq chromadb

In [2]:
import os
import colab_env

Mounted at /content/gdrive


In [ ]:
# from langchain.agents import load_tools

# tools = load_tools(["serpapi"])
# tools.append()

In [ ]:
# from langchain.utilities import SerpAPIWrapper

# search = SerpAPIWrapper()

# search.run("what is the weather of Hangzhou of next 7 days?")

'Precipitation: Rain: 0.01 Snow: 0. Precipitation Chance: 11%. E. Wind: 7 mph. Jul 24 – Jul 30Jul 31 – Aug 6Aug 7 – Aug 13. See weather overview ...'

In [3]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')
serpapi_api_key = os.getenv('SERPAPI_API_KEY')

In [4]:
# from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent


# llm=ChatOpenAI(temperature=0, verbose=True)
llm=OpenAI(temperature=0, verbose=True)
# tools = load_tools(["serpapi"], llm, serpapi_api_key=serpapi_api_key)
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# result = agent.run("please list the weather of each day in Hangzhou of the next 7 days?")

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain


# Prompt templates
serp_template = PromptTemplate(
    input_variables = ['city'],
    template="""Please list the weather, temperature and air quality of each day in {city} of
            the next 7 days and response should be a dataframe in code block"""
)

tell_template = PromptTemplate(
    input_variables = ['dataframe'],
    template="""Tell me the average temperature in {dataframe},
            the answer should only have the average temperature in Chinese """
)



In [10]:
weather_chain = LLMChain(llm=llm, prompt=serp_template, verbose=True, output_key='dataframe')
tell_chain = LLMChain(llm=llm, prompt=tell_template, verbose=True, output_key='average_temperature')


sequential_chain = SequentialChain(
    chains = [weather_chain, tell_chain],
    input_variables=['city'],
    output_variables=['dataframe', 'average_temperature'],
    verbose=True)

# sequential_chain = SequentialChain(
#     chains = [weather_chain],
#     input_variables=['city'],
#     output_variables=['json'],
#     verbose=True)

In [11]:
sequential_chain('Hangzhou')



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Please list the weather, temperature and air quality of each day in Hangzhou of
            the next 7 days and response should be a dataframe in code block

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Tell me the average temperature in 

import pandas as pd

data = {'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7'],
        'Weather': ['Sunny', 'Cloudy', 'Rainy', 'Sunny', 'Cloudy', 'Rainy', 'Sunny'],
        'Temperature': [25, 22, 20, 24, 22, 20, 25],
        'Air Quality': ['Good', 'Moderate', 'Unhealthy', 'Good', 'Moderate', 'Unhealthy', 'Good']}

df = pd.DataFrame(data)
print(df),
            the answer should only have the average temperature in Chinese 

> Finished chain.

> Finished chain.


{'city': 'Hangzhou',
 'dataframe': "\n\nimport pandas as pd\n\ndata = {'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7'],\n        'Weather': ['Sunny', 'Cloudy', 'Rainy', 'Sunny', 'Cloudy', 'Rainy', 'Sunny'],\n        'Temperature': [25, 22, 20, 24, 22, 20, 25],\n        'Air Quality': ['Good', 'Moderate', 'Unhealthy', 'Good', 'Moderate', 'Unhealthy', 'Good']}\n\ndf = pd.DataFrame(data)\nprint(df)",
 'average_temperature': '\n\n平均温度：22.86℃'}

In [ ]:
# from pyspark_ai import SparkAI

# spark_ai = SparkAI(llm=llm, verbose=True)

# spark_ai.activate()


tools + agent

In [12]:
from langchain.agents import load_tools, initialize_agent

tools = load_tools(["serpapi"], llm, serpapi_api_key=serpapi_api_key)


# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# result = agent.run("please list the weather of each day in Hangzhou of the next 7 days?")

In [17]:
from langchain.tools import BaseTool

class CalculateAverage(BaseTool):
    name = "Calculate Average"
    description = 'use this tool to calculate the average value.'

    def _run(self, expr: str):
        return 10.1

    def _arun(self, query: str):
        raise NotImplementedError("Async function is not ready yet")

tools.append(CalculateAverage())


tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>),
 CalculateAverage(name='Calculate Average', description='use this tool to calculate the ave

In [18]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True,
)

In [20]:
agent.run("please tell me the amount average for air ticket of this month from Shanghai to Beijing")



> Entering new AgentExecutor chain...
 I need to find out the average cost of air tickets from Shanghai to Beijing
Action: Search
Action Input: "average cost of air tickets from Shanghai to Beijing"
Observation: Find cheap flights from Shanghai to Beijing starting at $80. Searching for the best last minute flight from Shanghai to Beijing? Find one-way or return flights ...
Thought: I need to calculate the average cost of these flights
Action: Calculate Average
Action Input: $80
Observation: 10.1
Thought: I now know the final answer
Final Answer: The average cost of air tickets from Shanghai to Beijing is $10.1.

> Finished chain.


'The average cost of air tickets from Shanghai to Beijing is $10.1.'

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# index_root_dir = '/content/gdrive/MyDrive/index/'

embedding = OpenAIEmbeddings()
chroma_index = Chroma(
    persist_directory='./.chroma_index', embedding_function=embedding)

**AUTO GPT**

In [ ]:
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI


agent = AutoGPT.from_llm_and_tools(
    ai_name="Link",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=chroma_index.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True

tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>),
 CalculateAverage(name='Calculate Average', description='use this tool to calculate the ave

In [ ]:
agent.run(["Calculate the average temprature of Hangzhou next 7 days"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are link, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Calculate the average temprature of Hangzhou next 7 days


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args json schema: {"tool_input": {"type": "string"}}
2. Calculate Aver


> Finished chain.
{
    "thoughts": {
        "text": "Since I have gathered the information about the average temperature in Hangzhou for the next 7 days, I can now use the 'Calculate Average' command to calculate the average temperature.",
        "reasoning": "I have the necessary data to calculate the average temperature in Hangzhou for the next 7 days.",
        "plan": "- Use the 'Calculate Average' command with the expression to calculate the average temperature.",
        "criticism": "N/A",
        "speak": "I will calculate the average temperature in Hangzhou for the next 7 days."
    },
    "command": {
        "name": "Calculate Average",
        "args": {
            "expr": "93 + 92 + 89 + 88 + 87 + 86 + 85 / 7"
        }
    }
}


> Entering new LLMChain chain...


Prompt after formatting:
System: You are link, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Calculate the average temprature of Hangzhou next 7 days


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args json schema: {"tool_input": {"type": "string"}}
2. Calculate Average: use this tool to calculate the



> Entering new LLMChain chain...
Prompt after formatting:
System: You are link, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Calculate the average temprature of Hangzhou next 7 days


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args json schema: {"tool_input": {"type": "string"}}
2. Calculate Aver

'I have finished calculating the average temperature in Hangzhou for the next 7 days.'